@article{wagner2022dawn,
  title={Dawn of the transformer era in speech emotion recognition: closing the valence gap},
  author={Wagner, Johannes and Triantafyllopoulos, Andreas and Wierstorf, Hagen and Schmitt, Maximilian and Burkhardt, Felix and Eyben, Florian and Schuller, Bj{\"o}rn W.},
  journal={arXiv preprint arXiv:2203.07378},
  year={2022}
}

In [1]:
import audeer
import audonnx
import numpy as np

url = 'https://zenodo.org/record/6221127/files/w2v2-L-robust-12.6bc4a7fd-1.1.0.zip'
cache_root = audeer.mkdir('cache')
model_root = audeer.mkdir('model')

archive_path = audeer.download_url(url, cache_root, verbose=True)
audeer.extract_archive(archive_path, model_root)
model = audonnx.load(model_root)

sampling_rate = 16000
signal = np.random.normal(size=sampling_rate).astype(np.float32)
model(signal, sampling_rate)


{'hidden_states': array([[-0.00705336,  0.00607442, -0.0080355 , ...,  0.00673694,
          0.00923399,  0.00273004]], dtype=float32),
 'logits': array([[0.6573795 , 0.62606376, 0.48850167]], dtype=float32)}

In [2]:
signal

array([-0.01311579, -0.04658256,  3.4684708 , ...,  0.05826281,
       -0.30195874,  0.4372727 ], dtype=float32)

In [30]:
from datasets import load_dataset
import utils.audio_dataset_utils as audioUtils
import pandas as pd

test_dataset = load_dataset("csv", data_files={"validation": "content/data/train.csv"}, delimiter="\t")["validation"]

Using custom data configuration default-a7a609ab4d1c50a4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/ckwdani/.cache/huggingface/datasets/csv/default-a7a609ab4d1c50a4/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
def preprocess_function(examples):
    speech_list = [audioUtils.speech_file_to_array_fn(path, 16000) for path in examples["path"]]
    result = speech_list

    return result

In [32]:
import librosa
x = audioUtils.speech_file_to_array_librosa(test_dataset["path"][4], 16000)
i=40
y,sr = librosa.load(test_dataset["path"][i], sr=16000)
print(test_dataset["emotion"][i])

happy


In [33]:
test_dataset["path"][i]

'../tess/YAF_happy/YAF_yearn_happy.wav'

In [34]:
from torch import nn

#arousal, dominance, valence.
logits = model(y, 16000)
logits["logits"]

array([[0.6258381 , 0.68025964, 0.58995026]], dtype=float32)

In [7]:
test_dataset = test_dataset.map(
    preprocess_function,
    batch_size=50,
    batched=True
    #num_proc=4
)

  0%|          | 0/4 [00:00<?, ?ba/s]

TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.